In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as soup
import requests

In [35]:
movie_title = []
movie_rating = []
movie_genres = []
movie_url = []
movie_tags = []

rsub = requests.get('https://www.imdb.com/title/tt0087800/?ref_=adv_li_tt').content
datasub = soup(rsub,'html.parser')
tags_data = datasub.findAll('div',{'class':['see-more', 'inline', 'canwrap']})

reqd_div='None'
tags_list=[]
tags=[]
for t in tags_data:
    try:
        text = t.h4.text
        if text == 'Plot Keywords:':
            reqd_div = t
            tags_list = reqd_div.findAll('span',{'class':'itemprop'})
            for tl in tags_list:
                tags.append(tl.text)
    except:
        continue

print(tags)

In [36]:
def scrapeData(url):
    r = requests.get(url).content
    data = soup(r,'html.parser')
    movie_divs = data.findAll('div',{'class':'lister-item-content'})

    for div in movie_divs:
        #title
        t=div.h3.text.split()
        s=""
        index=1
        for i in t:
            if index==1:
                index+=1
                continue
            s+=i+" "
            index+=1
        movie_title.append(s)
    
        #genre
        g=div.findAll('span',{'class':'genre'})[0].text[1:]
        movie_genres.append(g)
    
        #rating
        try:
            rt = div.strong.text
        except:
            rt = 'None'
        movie_rating.append(rt)
        #url
        u = div.h3.a['href']
        u = 'www.imdb.com'+u
        movie_url.append(u)
        
        #tags
        rsub = requests.get('https://www.imdb.com/title/tt0087800/?ref_=adv_li_tt').content
        datasub = soup(rsub,'html.parser')
        tags_data = datasub.findAll('div',{'class':['see-more', 'inline', 'canwrap']})

        reqd_div='None'
        tags_list=[]
        tags=[]
        for t in tags_data:
            try:
                text = t.h4.text
                if text == 'Plot Keywords:':
                    reqd_div = t
                    tags_list = reqd_div.findAll('span',{'class':'itemprop'})
                    for tl in tags_list:
                        tags.append(tl.text)
            except:
                tags.append([])
                continue
        movie_tags.append(tags)

In [37]:
#scrapeData('https://www.imdb.com/search/title/?title_type=feature&genres=horror&start=51&explore=genres&ref_=adv_nxt')

In [38]:
def getNextURL(url):
    r = requests.get(url).content
    data = soup(r,'html.parser')
    div = data.find('a',{'class':'next-page'})['href']
    nextUrl = 'https://www.imdb.com/'+div
    return nextUrl    

In [39]:
#getNextURL('https://www.imdb.com/search/title/?genres=horror&title_type=feature&explore=genres')

In [29]:
start_url = "https://www.imdb.com/search/title/?genres=horror&title_type=feature&explore=genres"
pageNo = 1

while pageNo<=50:
    scrapeData(start_url)
    start_url = getNextURL(start_url)
    print("Done scraping Page no:" +pageNo)
    pageNo+=1               

KeyboardInterrupt: 

In [40]:
#print(movie_title)

['serial killer', 'slasher', 'freddy krueger', 'sleep deprivation', 'murder']


### Data to csv

In [ ]:
print(len(movie_title))
print(len(movie_url))
print(len(movie_rating))
print(len(movie_genres))
print(len(movie_tags))

In [122]:
df = pd.DataFrame({'Title':movie_title,'Rating':movie_rating,'Genres':movie_genres,'Tags':movie_tags,'Url':movie_url})

In [123]:
#df.to_csv('Horror Movies.csv',index=False)
df.to_csv('Horror Movies with Tags.csv',index=False)